In [3]:
def draw_Kline(stock_code,save_name,save_dir):
    test_file = "%s/%s.csv" % (history_data_dir, stock_code)
    df = pd.read_csv(test_file)
    df.rename(columns={"vol":"volume"},inplace=True)
    df["trade_date"] = pd.to_datetime(df["trade_date"],format='%Y%m%d')
    df.set_index("trade_date", inplace=True)

    # 符合中国习惯的配色风格——红涨绿跌自然是必须实现的第一步
    my_color = mpl.make_marketcolors(up='r',
                                     down='g',
                                     edge='inherit',
                                     wick='inherit',
                                     volume='inherit')
    # 设置图表的背景色
    my_style = mpl.make_mpf_style(marketcolors=my_color,
                                  figcolor='(0.82, 0.83, 0.85)',
                                  gridcolor='(0.82, 0.83, 0.85)')

    df["diff"],df["dem"],df["macd_volume"] = macd(df)
    df["ma120"] = ma(df,n=120)
    df["ma250"] = ma(df,n=250)
    plot_df = df.iloc[-120:,]
    bar_r = np.where(plot_df['macd_volume'] > 0, plot_df['macd_volume'], 0)
    add_plot = [mpl.make_addplot(plot_df[["diff","dem"]],panel=2),
                mpl.make_addplot(plot_df["ma120"], color="grey",panel=0),
                mpl.make_addplot(plot_df["ma250"], color="blue",panel=0),
                mpl.make_addplot(bar_r, type='bar',color="red",panel=2)]
    save = '%s/%s_%s.jpg' %(save_dir,stock_code,save_name)
    mpl.plot(plot_df, type = 'candle',style=my_style,figsize=(12, 8),title=save_name,addplot=add_plot,
                 mav = (5,10,20, 30, 60),volume = True, savefig=save)